In [1]:
import os, re, sys
from scripts.tools.issuetracker import IssuetrackerAPI
from datetime import date
from officelib.xllib import *
from jpnotebooks.Software.SDLC_traceability_tools.issuetracker_item_extracter import RequirementExtracter

In [9]:
_cache = None
_api = None
_sync_date = None
def _download(**kw):
    global _cache, _api, _sync_date
    if _api is None:
        _api = IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
    _cache = _api.download_issues(project_id="pbssoftware", **kw)
    _sync_date = date.today()
    
def download(force=False, **kw):
    global _cache, _sync_date
    reason = None
    if _cache is None:
        reason = "cache is None"
    elif force:
        reason = "forced re-cache"
    elif _sync_date is None:
        reason = "can't determine last sync time"
    elif date.today() != _sync_date:
        reason = "cache age > 0 days"
    if reason:
        print("Downloading issues:", reason)
        _download(**kw)
    return _cache.copy()

def load_issues(**kw):
    return download(**kw)

In [13]:
tags = [
    "URS",
    "FRS",
    "SDS",
]
issues = load_issues()
#issues = list(issues.values())
rows = RequirementExtracter(tags).extract_text(issues[3646].description)

In [14]:
def paste(cr, c1, data):
    c2 = c1.Offset(len(data), len(data[0]))
    cr(c1, c2).Value = data

In [15]:
xl, wb = xlBook2()
ws = wb.Worksheets(1)
cells = ws.Cells
cr = cells.Range
paste(cr, cells(1,1), rows)